# Gastric Warriors Differential gene expression analysis



Importing correct files: Using bash


In [20]:
%%bash

workdir="/cephyr/NOBACKUP/groups/bbt045_2025/groups/group_gaswar/Data/"
echo "Working directory set to $workdir"

RNAlist="/cephyr/NOBACKUP/groups/bbt045_2025/Projects/Data/RNA-seq-list.txt"
RNAdata="/cephyr/NOBACKUP/groups/bbt045_2025/Projects/Data/RNAseq/"
echo "RNAseq-data in ${RNAdata}"


grep "Met" $RNAlist | cut -f 1 > Metaplasia_patients.txt
grep "Hp" $RNAlist | cut -f 1 >> Metaplasia_patients.txt

#find $RNAdata -type f -name "*subset.fastq*" | grep -Ff Metaplasia_patients.txt | xargs -I {} cp {} "$workdir"

/usr/share/lmod/lmod/init/bash: /usr/share/lmod/lmod/libexec/addto: /usr/bin/lua: bad interpreter: No such file or directory


Working directory set to /cephyr/NOBACKUP/groups/bbt045_2025/groups/group_gaswar/Data/
RNAseq-data in /cephyr/NOBACKUP/groups/bbt045_2025/Projects/Data/RNAseq/


Create a CSV with patient list and health condition

In [5]:
%%bash

#Create patient list
RNAlist="/cephyr/NOBACKUP/groups/bbt045_2025/Projects/Data/RNA-seq-list.txt"

# Define the output file
OUTPUT="patient_list.csv"
# Add CSV header
echo "PatientNumber,HealthCondition" > $OUTPUT

# Create patient_list.txt with "Met" and "Hp" conditions
awk '{print $1, ",Met"}' <(grep "Met" $RNAlist | cut -f1) >> $OUTPUT
awk '{print $1, ",Hp"}' <(grep "Hp" $RNAlist | cut -f1) >> $OUTPUT


/usr/share/lmod/lmod/init/bash: /usr/share/lmod/lmod/libexec/addto: /usr/bin/lua: bad interpreter: No such file or directory


Now decompress all the files

In [25]:
%%bash
#set the working directory
workdir="/cephyr/NOBACKUP/groups/bbt045_2025/groups/group_gaswar/Data/"

#use gzip to decrompress the files
gzip -d "${workdir}"*


/usr/share/lmod/lmod/init/bash: /usr/share/lmod/lmod/libexec/addto: /usr/bin/lua: bad interpreter: No such file or directory


# MultiQC

In [2]:
#First generate fastQC reports, all files in Data and output to /subset_fastqc/
! fastqc Data/*.subset.fastq -o subset_fastqc/

/usr/share/lmod/lmod/init/bash: /usr/share/lmod/lmod/libexec/addto: /usr/bin/lua: bad interpreter: No such file or directory
null
null
Started analysis of ERR950168_1.fastq.gz.subset.fastq
null
null
null
null
null
null
null
null
null
null
null
Approx 5% complete for ERR950168_1.fastq.gz.subset.fastq
null
null
null
null
null
Approx 10% complete for ERR950168_1.fastq.gz.subset.fastq
Approx 15% complete for ERR950168_1.fastq.gz.subset.fastq
Approx 20% complete for ERR950168_1.fastq.gz.subset.fastq
Approx 25% complete for ERR950168_1.fastq.gz.subset.fastq
Approx 30% complete for ERR950168_1.fastq.gz.subset.fastq
Approx 35% complete for ERR950168_1.fastq.gz.subset.fastq
Approx 40% complete for ERR950168_1.fastq.gz.subset.fastq
Approx 45% complete for ERR950168_1.fastq.gz.subset.fastq
Approx 50% complete for ERR950168_1.fastq.gz.subset.fastq
Approx 55% complete for ERR950168_1.fastq.gz.subset.fastq
Approx 60% complete for ERR950168_1.fastq.gz.subset.fastq
Approx 65% complete for ERR950168_1.

In [7]:
#Take entire subset_fastqc/ and generate multiqc-file
! multiqc subset_fastqc/ -n multiqc_pre_subset.html

/usr/share/lmod/lmod/init/bash: /usr/share/lmod/lmod/libexec/addto: /usr/bin/lua: bad interpreter: No such file or directory

/// ]8;id=395354;https://multiqc.info\MultiQC]8;;\ 🔍 v1.27.1

       file_search | Search path: /cephyr/NOBACKUP/groups/bbt045_2025/groups/group_gaswar/subset_fastqc
         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 36/36  pm  
            fastqc | Found 18 reports
     write_results | Data        : multiqc_pre_subset_data
     write_results | Report      : multiqc_pre_subset.html
           multiqc | MultiQC complete


# Preprocessing using FastP
> 

In [10]:
%%bash

#Output
mkdir -p trimmed_fastq_subset


# Loop through all forward reads (_R1.fastq.gz.subset.fastq) and find their corresponding reverse reads (_R2.fastq.gz.subset.fastq)
for R1 in Data/*_1.fastq.gz.subset.fastq; do
    # Extract the sample name by removing _R1.fastq.gz.subset.fastq
    SAMPLE=$(basename "$R1" _1.fastq.gz.subset.fastq)
    # Define file names
    R2="Data/${SAMPLE}_2.fastq.gz.subset.fastq"         # Reverse read file

    echo "Checking: $R1 and $R2"

    OUT_R1="trimmed_fastq_subset/${SAMPLE}_1_trimmed_subset.fastq.gz"
    OUT_R2="trimmed_fastq_subset/${SAMPLE}_2_trimmed_subset.fastq.gz"
    # Run fastp
    fastp -i "$R1" -I "$R2" -o "$OUT_R1" -O "$OUT_R2" --qualified_quality_phred 30

    echo "Finished processing: $SAMPLE"
done

/usr/share/lmod/lmod/init/bash: /usr/share/lmod/lmod/libexec/addto: /usr/bin/lua: bad interpreter: No such file or directory


Checking: Data/ERR950168_1.fastq.gz.subset.fastq and Data/ERR950168_2.fastq.gz.subset.fastq


Read1 before filtering:
total reads: 500000
total bases: 50500000
Q20 bases: 49033765(97.0966%)
Q30 bases: 46201084(91.4873%)

Read2 before filtering:
total reads: 500000
total bases: 50500000
Q20 bases: 48644322(96.3254%)
Q30 bases: 46586827(92.2511%)

Read1 after filtering:
total reads: 471900
total bases: 45504357
Q20 bases: 44769741(98.3856%)
Q30 bases: 42633528(93.6911%)

Read2 after filtering:
total reads: 471900
total bases: 45504357
Q20 bases: 44928366(98.7342%)
Q30 bases: 43452519(95.4909%)

Filtering result:
reads passed filter: 943800
reads failed due to low quality: 53744
reads failed due to too many N: 2456
reads failed due to too short: 0
reads with adapter trimmed: 201558
bases trimmed due to adapters: 4412742

Duplication rate: 15.1342%

Insert size peak (evaluated by paired-end reads): 124

JSON report: fastp.json
HTML report: fastp.html

fastp -i Data/ERR950168_1.fastq.gz.subset.fastq -I Data/ERR950168_2.fastq.gz.subset.fastq -o trimmed_fastq_subset/ERR950168_1_trimme

Finished processing: ERR950168
Checking: Data/ERR950169_1.fastq.gz.subset.fastq and Data/ERR950169_2.fastq.gz.subset.fastq


Read1 before filtering:
total reads: 500000
total bases: 50500000
Q20 bases: 45991156(91.0716%)
Q30 bases: 40593825(80.3838%)

Read2 before filtering:
total reads: 500000
total bases: 50500000
Q20 bases: 48440432(95.9216%)
Q30 bases: 46774237(92.6223%)

Read1 after filtering:
total reads: 455588
total bases: 43605416
Q20 bases: 40825536(93.6249%)
Q30 bases: 36693088(84.148%)

Read2 after filtering:
total reads: 455588
total bases: 43605416
Q20 bases: 43225131(99.1279%)
Q30 bases: 42166906(96.7011%)

Filtering result:
reads passed filter: 911176
reads failed due to low quality: 87002
reads failed due to too many N: 1822
reads failed due to too short: 0
reads with adapter trimmed: 238964
bases trimmed due to adapters: 4961722

Duplication rate: 10.1618%

Insert size peak (evaluated by paired-end reads): 124

JSON report: fastp.json
HTML report: fastp.html

fastp -i Data/ERR950169_1.fastq.gz.subset.fastq -I Data/ERR950169_2.fastq.gz.subset.fastq -o trimmed_fastq_subset/ERR950169_1_trimmed

...

Finished processing: ERR950185
Checking: Data/ERR950186_1.fastq.gz.subset.fastq and Data/ERR950186_2.fastq.gz.subset.fastq


Read1 before filtering:
total reads: 500000
total bases: 50500000
Q20 bases: 46197465(91.4801%)
Q30 bases: 40838552(80.8684%)

Read2 before filtering:
total reads: 500000
total bases: 50500000
Q20 bases: 48655120(96.3468%)
Q30 bases: 46913618(92.8983%)

Read1 after filtering:
total reads: 459045
total bases: 44515737
Q20 bases: 41690952(93.6544%)
Q30 bases: 37398979(84.0129%)

Read2 after filtering:
total reads: 459045
total bases: 44515737
Q20 bases: 44107438(99.0828%)
Q30 bases: 42937108(96.4538%)

Filtering result:
reads passed filter: 918090
reads failed due to low quality: 80044
reads failed due to too many N: 1866
reads failed due to too short: 0
reads with adapter trimmed: 174986
bases trimmed due to adapters: 3780002

Duplication rate: 13.6372%

Insert size peak (evaluated by paired-end reads): 124

JSON report: fastp.json
HTML report: fastp.html

fastp -i Data/ERR950186_1.fastq.gz.subset.fastq -I Data/ERR950186_2.fastq.gz.subset.fastq -o trimmed_fastq_subset/ERR950186_1_trimme

Finished processing: ERR950186
Checking: Data/ERR950187_1.fastq.gz.subset.fastq and Data/ERR950187_2.fastq.gz.subset.fastq


Read1 before filtering:
total reads: 500000
total bases: 50500000
Q20 bases: 48869499(96.7713%)
Q30 bases: 45832198(90.7568%)

Read2 before filtering:
total reads: 500000
total bases: 50500000
Q20 bases: 48336303(95.7155%)
Q30 bases: 46116041(91.3189%)

Read1 after filtering:
total reads: 467050
total bases: 45717950
Q20 bases: 44879626(98.1663%)
Q30 bases: 42512897(92.9895%)

Read2 after filtering:
total reads: 467050
total bases: 45717950
Q20 bases: 45024034(98.4822%)
Q30 bases: 43380186(94.8866%)

Filtering result:
reads passed filter: 934100
reads failed due to low quality: 63604
reads failed due to too many N: 2296
reads failed due to too short: 0
reads with adapter trimmed: 149058
bases trimmed due to adapters: 2985616

Duplication rate: 20.043%

Insert size peak (evaluated by paired-end reads): 124

JSON report: fastp.json
HTML report: fastp.html

fastp -i Data/ERR950187_1.fastq.gz.subset.fastq -I Data/ERR950187_2.fastq.gz.subset.fastq -o trimmed_fastq_subset/ERR950187_1_trimmed

Finished processing: ERR950187


In [11]:
#Fastqc of the fastp dataset
! mkdir -p FastQC_aftertrim_subset
! fastqc trimmed_fastq_subset/*_subset.fastq.gz -o FastQC_aftertrim_subset/
! multiqc FastQC_aftertrim_subset/ -n aftertrim_multiqc_subset.html

/usr/share/lmod/lmod/init/bash: /usr/share/lmod/lmod/libexec/addto: /usr/bin/lua: bad interpreter: No such file or directory
/usr/share/lmod/lmod/init/bash: /usr/share/lmod/lmod/libexec/addto: /usr/bin/lua: bad interpreter: No such file or directory
application/gzip
application/gzip
Started analysis of ERR950168_1_trimmed_subset.fastq.gz
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
application/gzip
Approx 5% complete for ERR950168_1_trimmed_subset.fastq.gz
Approx 10% complete for ERR950168_1_trimmed_subset.fastq.gz
Approx 15% complete for ERR950168_1_trimmed_subset.fastq.gz
Approx 20% complete for ERR950168_1_trimmed_subset.fastq.gz
Approx 25% complete for ERR950168_1_trimmed_subset.fastq.gz
Approx 30% complete for ERR950168_1_trimmed_subset.fastq.gz
Approx 35% complete for ERR95

In [6]:
import pandas as pd

patient_list = pd.read_csv(patient_list.csv)
print(patient_list)

ModuleNotFoundError: No module named 'pandas'